In [31]:
# pip install seaborn
!pip install scikit-learn

     ---------------------------------------- 9.3/9.3 MB 59.3 MB/s eta 0:00:00
     ---------------------------------------- 302.0/302.0 KB ? eta 0:00:00
     --------------------------------------- 44.1/44.1 MB 23.4 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

# ml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
train = pd.read_csv('./datas/train.csv')
test = pd.read_csv('./datas/test.csv')
building = pd.read_csv('./datas/building_info.csv')


In [17]:
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train.drop('num_date_time', axis = 1, inplace=True)

In [18]:
test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test.drop('num_date_time', axis = 1, inplace=True)

In [19]:
building = building.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'floor_area',
    '냉방면적(m2)': 'cooling Area',
    '태양광용량(kW)': 'solar_capacity',
    'ESS저장용량(kWh)': 'ESS_capacity',
    'PCS용량(kW)': 'PCS_capacity'
})
building = building.replace('-', 0)

In [20]:
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building['building_type'] = building['building_type'].replace(translation_dict)


In [21]:
building.head()

,building_number,building_type,floor_area,cooling Area,solar_capacity,ESS_capacity,PCS_capacity
0,1,Other Buildings,110634.00,39570.00,0,0,0
1,2,Other Buildings,122233.47,99000.00,0,0,0
2,3,Other Buildings,171243.00,113950.00,40,0,0
3,4,Other Buildings,74312.98,34419.62,60,0,0
4,5,Other Buildings,205884.00,150000.00,0,2557,1000


In [22]:
test.head()

,building_number,date_time,temperature,rainfall,windspeed,humidity
0,1,20220825 00,23.5,0.0,2.2,72
1,1,20220825 01,23.0,0.0,0.9,72
2,1,20220825 02,22.7,0.0,1.5,75
3,1,20220825 03,22.1,0.0,1.3,78
4,1,20220825 04,21.8,0.0,1.0,77


In [23]:
# 결측치
## train
train['rainfall'] = train['rainfall'].fillna(0)
train['sunshine'] = train['sunshine'].fillna(0)
train['solar_radiation'] = train['solar_radiation'].fillna(0)
train['windspeed'] = train['windspeed'].fillna(train['windspeed'].mean())
train['humidity'] = train['humidity'].fillna(train['humidity'].mean())
## test
test['rainfall'] = test['rainfall'].fillna(0)
test['sunshine'] = 0
test['solar_radiation'] = 0
test['humidity'] = test['humidity'].fillna(test['humidity'].mean())
test['windspeed'] = test['windspeed'].fillna(test['windspeed'].mean())

# 날짜 파생 변수

## train
train['date_time'] = pd.to_datetime(train['date_time'], format = '%Y%m%d %H')
train['year'] = train['date_time'].dt.year
train['month'] = train['date_time'].dt.month
train['day'] = train['date_time'].dt.day
train['hour'] = train['date_time'].dt.hour
train['weekday'] = train['date_time'].dt.weekday
train['weekend'] = train['weekday'].isin([5,6]).astype(int)
train.drop(columns ='date_time', axis = 1, inplace = True)
## test
test['date_time'] = pd.to_datetime(test['date_time'], format = '%Y%m%d %H')
test['year'] = test['date_time'].dt.year
test['month'] = test['date_time'].dt.month
test['day'] = test['date_time'].dt.day
test['hour'] = test['date_time'].dt.hour
test['weekday'] = test['date_time'].dt.weekday
test['weekend'] = test['weekday'].isin([5,6]).astype(int)
test.drop(columns ='date_time', axis = 1, inplace = True)

In [ ]:
train.head()

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   building_number    204000 non-null  int64  
 1   temperature        204000 non-null  float64
 2   rainfall           204000 non-null  float64
 3   windspeed          204000 non-null  float64
 4   humidity           204000 non-null  float64
 5   sunshine           204000 non-null  float64
 6   solar_radiation    204000 non-null  float64
 7   power_consumption  204000 non-null  float64
 8   year               204000 non-null  int32  
 9   month              204000 non-null  int32  
 10  day                204000 non-null  int32  
 11  hour               204000 non-null  int32  
dtypes: float64(7), int32(4), int64(1)
memory usage: 15.6 MB


In [27]:
train.describe()

,building_number,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,year,month,day,hour
count,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.0,204000.000000,204000.000000,204000.000000
mean,50.500000,25.543524,0.439489,2.134031,78.716295,0.210679,0.605373,2451.036462,2022.0,6.929412,14.835294,11.500000
std,28.866141,3.779523,2.539497,1.358045,15.079775,0.357338,0.920739,2440.648858,0.0,0.793923,8.446178,6.922204
min,1.000000,10.100000,0.000000,0.000000,13.000000,0.000000,0.000000,0.000000,2022.0,6.000000,1.000000,0.000000
25%,25.750000,23.400000,0.000000,1.200000,69.000000,0.000000,0.000000,1085.760000,2022.0,6.000000,8.000000,5.750000
50%,50.500000,25.700000,0.000000,2.000000,81.000000,0.000000,0.040000,1766.400000,2022.0,7.000000,15.000000,11.500000
75%,75.250000,28.100000,0.000000,2.800000,91.000000,0.300000,0.930000,2970.000000,2022.0,8.000000,22.000000,17.250000
max,100.000000,37.100000,92.200000,13.300000,100.000000,1.000000,3.920000,25488.400000,2022.0,8.000000,31.000000,23.000000


In [34]:
X_train = train.drop(columns='power_consumption')
y_train = train['power_consumption']

In [35]:
X_train.head()

,building_number,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,year,month,day,hour
0,1,18.6,0.0,0.9,42.0,0.0,0.0,2022,6,1,0
1,1,18.0,0.0,1.1,45.0,0.0,0.0,2022,6,1,1
2,1,17.7,0.0,1.5,45.0,0.0,0.0,2022,6,1,2
3,1,16.7,0.0,1.4,48.0,0.0,0.0,2022,6,1,3
4,1,18.4,0.0,2.8,43.0,0.0,0.0,2022,6,1,4


In [ ]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

https://dacon.io/competitions/official/235736/codeshare/2877?page=1&dtype=recent

https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent